In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

import pandas as pd
import numpy as np

import sklearn as skl
from sklearn.svm import SVC
from sklearn.base import BaseEstimator
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from tensorflow.keras import Input,Model
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import plot_model

In [ ]:
data = pd.read_csv("./data/monodata/car evaluation.csv",header=None)
features = len(data.columns)-1
data.columns = [f"crit_{i}" for i in range(features)]+["class"]
data

Binerazing the classes

In [ ]:
labels = pd.get_dummies(data["class"])
labels.columns = [f"class_{int(i)}" for i in labels.columns]
labels = labels.astype(float)
data_classless = data.drop(columns="class")
labels

Split into train and test data

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data_classless.to_numpy(),labels.to_numpy(),test_size=0.20)

## RankSVM method

### Calculate differencese between rows of different classes.

In [ ]:
def calculate_differences(array:np.array):
    difs = array[:,np.newaxis,:] - array 
    return difs.reshape(-1,array.shape[1])

In [ ]:
X_train_difs = calculate_differences(X_train)
X_test_difs = calculate_differences(X_test)

y_train_difs = calculate_differences(y_train)
y_test_difs = calculate_differences(y_test)

In [ ]:
train_sums = np.any(y_train_difs!=0,axis=1)
test_sums = np.any(y_test_difs!=0,axis=1)

y_train_filtered = y_train_difs[train_sums]
y_test_filtered = y_test_difs[test_sums]

X_train_filtered = X_train_difs[train_sums]
X_test_filtered = X_test_difs[test_sums]

In [ ]:
X_train_filtered.shape

In [ ]:
rank_svm = make_pipeline(StandardScaler(),SVC())

#rank_svm.fit_transform(X_train_filtered,y_train_filtered)

## ANN-UTADIS

## Neural Network

In [ ]:
inputs = Input((features,))
layer = Dense(128,activation="relu")(inputs)
layer = Dense(64,activation="relu")(layer)
layer = Dense(32,activation="relu")(layer)
outputs = Dense(4,activation="sigmoid")(layer)

model = Model(inputs=inputs,outputs=outputs)

In [ ]:
model.summary()

In [ ]:
plot_model(model)

In [ ]:
model.compile(loss="mae")
model.fit(X_train,y_train)